In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.10.10 데이터 추출 완료 - 31140 데이터가 대부분 없음, InterfaceError: Python type ResultSet cannot be converted > updowncd 값 2에 의한 에러
cursor = mydb.cursor(prepared=True)
citycode = '37030'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'
            elif updowncd[0].text == '2':
                updowncd = '2'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20211010 | location : 36420 | cnt : 1 | start MPB339000001 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 2 | start MPB339000004 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 3 | start MPB339000005 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 4 | start MPB339000010 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 5 | start MPB339000015 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 6 | start MPB339000017 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 7 | start MPB339000020 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 8 | start MPB339000022 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 9 | start MPB339000024 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 10 | start MPB339000031 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 11 | start MPB339000033 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt 

조회날짜 : 20211010 | location : 36420 | cnt : 96 | start MPB339000115 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 97 | start MPB339000116 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 98 | start MPB339000117 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 99 | start MPB339000119 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 100 | start MPB339000121 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 101 | start MPB339000122 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 102 | start MPB339000123 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 103 | start MPB339000124 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 104 | start MPB339000125 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 105 | start MPB339000050 public_busroute_info
조회날짜 : 20211010 | location : 36420 | cnt : 106 | start MPB339000055 public_busroute_info
조회날짜 : 20211010 | locatio

조회날짜 : 20211010 | location : 36430 | cnt : 91 | start HPB60 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 1 | start TSB341000006 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 2 | start TSB341000007 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 3 | start TSB341000008 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 4 | start TSB341000009 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 5 | start TSB341000010 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 6 | start TSB341000011 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 7 | start TSB341000012 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 8 | start TSB341000013 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 9 | start TSB341000014 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 10 | start TSB341000015 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 11 | 

조회날짜 : 20211010 | location : 36440 | cnt : 95 | start TSB341000129 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 96 | start TSB341000130 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 97 | start TSB341000131 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 98 | start TSB341000132 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 99 | start TSB341000133 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 100 | start TSB341000134 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 101 | start TSB341000135 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 102 | start TSB341000136 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 103 | start TSB341000137 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 104 | start TSB341000138 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 105 | start TSB341000139 public_busroute_info
조회날짜 : 20211010 | location

조회날짜 : 20211010 | location : 36440 | cnt : 189 | start TSB341000152 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 190 | start TSB341000153 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 191 | start TSB341000154 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 192 | start TSB341000157 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 193 | start TSB341000158 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 194 | start TSB341000159 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 195 | start TSB341000163 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 196 | start TSB341000164 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 197 | start TSB341000168 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 198 | start TSB341000172 public_busroute_info
조회날짜 : 20211010 | location : 36440 | cnt : 199 | start TSB341000191 public_busroute_info
조회날짜 : 20211010 | loc

...

조회날짜 : 20211010 | location : 37010 | cnt : 150 | start PHB350000039 public_busroute_info
조회날짜 : 20211010 | location : 37010 | cnt : 151 | start PHB350000209 public_busroute_info
조회날짜 : 20211010 | location : 37010 | cnt : 152 | start PHB350000238 public_busroute_info
조회날짜 : 20211010 | location : 37010 | cnt : 153 | start PHB350000239 public_busroute_info
조회날짜 : 20211010 | location : 37010 | cnt : 154 | start PHB350000274 public_busroute_info
조회날짜 : 20211010 | location : 37010 | cnt : 155 | start PHB350000275 public_busroute_info
조회날짜 : 20211010 | location : 37010 | cnt : 156 | start PHB350000375 public_busroute_info
조회날짜 : 20211010 | location : 37010 | cnt : 157 | start PHB350000035 public_busroute_info
조회날짜 : 20211010 | location : 37010 | cnt : 158 | start PHB350000037 public_busroute_info
조회날짜 : 20211010 | location : 37010 | cnt : 159 | start PHB350000038 public_busroute_info
조회날짜 : 20211010 | location : 37010 | cnt : 160 | start PHB350000240 public_busroute_info
조회날짜 : 20211010 | loc

조회날짜 : 20211010 | location : 37020 | cnt : 57 | start KUB352000307 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 58 | start KUB352000319 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 59 | start KUB352000320 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 60 | start KUB352000324 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 61 | start KUB352000325 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 62 | start KUB352000326 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 63 | start KUB352000327 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 64 | start KUB352000328 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 65 | start KUB352000329 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 66 | start KUB352000330 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 67 | start KUB352000331 public_busroute_info
조회날짜 : 20211010 | location : 370

조회날짜 : 20211010 | location : 37020 | cnt : 151 | start KUB352000212 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 152 | start KUB352000213 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 153 | start KUB352000214 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 154 | start KUB352000215 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 155 | start KUB352000216 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 156 | start KUB352000217 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 157 | start KUB352000218 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 158 | start KUB352000219 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 159 | start KUB352000220 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 160 | start KUB352000221 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 161 | start KUB352000222 public_busroute_info
조회날짜 : 20211010 | loc

조회날짜 : 20211010 | location : 37020 | cnt : 244 | start KUB352000312 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 245 | start KUB352000313 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 246 | start KUB352000314 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 247 | start KUB352000317 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 248 | start KUB352000321 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 249 | start KUB352000322 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 250 | start KUB352000323 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 251 | start KUB352000012 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 252 | start KUB352000013 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 253 | start KUB352000089 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 254 | start KUB352000096 public_busroute_info
조회날짜 : 20211010 | loc

조회날짜 : 20211010 | location : 37020 | cnt : 337 | start KUB352000182 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 338 | start KUB352000183 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 339 | start KUB352000184 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 340 | start KUB352000024 public_busroute_info
조회날짜 : 20211010 | location : 37020 | cnt : 341 | start KUB352000146 public_busroute_info
조회날짜 : 20211010 | location : 37030 | cnt : 1 | start GCB47201252 public_busroute_info
조회날짜 : 20211010 | location : 37030 | cnt : 2 | start GCB47201253 public_busroute_info
조회날짜 : 20211010 | location : 37030 | cnt : 3 | start GCB47201254 public_busroute_info
조회날짜 : 20211010 | location : 37030 | cnt : 4 | start GCB47201255 public_busroute_info
조회날짜 : 20211010 | location : 37030 | cnt : 5 | start GCB47201256 public_busroute_info
조회날짜 : 20211010 | location : 37030 | cnt : 6 | start GCB47201257 public_busroute_info
조회날짜 : 20211010 | location : 37030 | cn